In [1]:
import requests
import pandas as pd
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from db_connection import start_engine
from datetime import date

engine = start_engine()

In [4]:
LAT = 53.5753
LON = 10.0153
years = range(2015,2026)

API_URL = "https://air-quality-api.open-meteo.com/v1/air-quality"

In [ ]:
def air_pollution(year):
    today = date.today()
    if year == today.year:
        end_date = today.strftime("%Y-%m-%d")
    else:
        end_date = f"{year}-12-31"
    params = {
        "latitude": LAT,
        "longitude": LON,
        "hourly": "pm10,pm2_5,nitrogen_dioxide,ozone,carbon_monoxide,sulphur_dioxide",
        "start_date": f"{year}-01-01",
        "end_date": end_date,
    }

    data = requests.get(API_URL, params=params).json()

    df = pd.DataFrame({
        "timestamp": data["hourly"]["time"],
        "pm10": data["hourly"]["pm10"],
        "pm2_5": data["hourly"]["pm2_5"],
        "no2": data["hourly"]["nitrogen_dioxide"],
        "o3": data["hourly"]["ozone"],
        "co": data["hourly"]["carbon_monoxide"],
        "so2": data["hourly"]["sulphur_dioxide"],
    })

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    return df


In [ ]:
years_data = []
for n in years:
    print(n)
    new_data = air_pollution(n, f"air_quality_{n}")
    years_data.append(new_data)
all_years_data = pd.concat(years_data, ignore_index=True)


2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025


/var/folders/4_/7rpxmwzd0q99rfstfcs03h400000gn/T/ipykernel_26252/1019232981.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_years_data = pd.concat(years_data, ignore_index=True)


In [15]:
all_years_data.to_sql(name="air_quality_hamburg_2015_2025", con=engine,
                       if_exists="replace", index=False)

712

In [ ]:
def air_pollution_by_bezirk(year, bezirk, lat, lon):
    today = date.today()

    if year == today.year:
        end_date = today.strftime("%Y-%m-%d")
    else:
        end_date = f"{year}-12-31"

    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "pm10,pm2_5,nitrogen_dioxide,ozone,carbon_monoxide,sulphur_dioxide",
        "start_date": f"{year}-01-01",
        "end_date": end_date,
    }

    raw = requests.get(API_URL, params=params).json()

    df = pd.DataFrame({
        "timestamp": raw["hourly"]["time"],
        "pm10": raw["hourly"]["pm10"],
        "pm2_5": raw["hourly"]["pm2_5"],
        "no2": raw["hourly"]["nitrogen_dioxide"],
        "o3": raw["hourly"]["ozone"],
        "co": raw["hourly"]["carbon_monoxide"],
        "so2": raw["hourly"]["sulphur_dioxide"],
    })

    df["year"] = year
    df["bezirk"] = bezirk

    return df

In [ ]:

stations = {
    "Hamburg-Mitte":     (53.5753, 10.0153),   # Habichtstraße / Stadthausbrücke
    "Altona":            (53.5542, 9.9350),    # Max-Brauer-Allee
    "Eimsbüttel":        (53.5900, 9.9640),    # Eimsbüttel station
    "Hamburg-Nord":      (53.6050, 10.0280),   # Winterhude / Hohenfelde
    "Wandsbek":          (53.6100, 10.1000),   # Steilshoop
    "Bergedorf":         (53.4840, 10.2140),   # Bergedorf
    "Harburg":           (53.4620, 9.9700)     # Neugraben
}

years = range(2023, 2026)
all_data = []

for bezirk, (lat, lon) in stations.items():
    print(f"Downloading {bezirk}")

    for year in years:
        df = air_pollution(year, bezirk, lat, lon)
        all_data.append(df)

final_df = pd.concat(all_data, ignore_index=True)


In [49]:
final_df.to_sql(name = "air_quality_bezirk_2023_2025", con= engine, # Push to DB
                       if_exists= "replace", index = False)

88